In [1]:
import pandas as pd
import numpy as np
import random
import implicit
from tqdm.notebook import tqdm
from scipy.sparse import coo_matrix, csr_matrix
from implicit.nearest_neighbours import BM25Recommender, CosineRecommender, TFIDFRecommender
import copy
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score
import lightgbm as lgb

In [2]:
data = pd.read_csv('dataset11-30.csv')
train_1 = pd.read_csv('export_arh_11-20-final.csv')
train_2 = pd.read_csv('export_arh_21-30-final.csv')
train_3 = pd.read_csv('export_arh_31-42-final.csv')

tv_schedule = pd.concat([train_1, train_2, train_3], axis = 0).reset_index(drop = True)

subm = pd.read_csv('submission.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
subm.user_id = subm.user_id.astype(str)

In [4]:
tv_schedule.tv_show_id = tv_schedule.tv_show_id.astype(str)
tv_schedule.channel_id = tv_schedule.channel_id.astype(str)

In [5]:
data.user_id = data.user_id.astype(str)
data.vsetv_id = data.vsetv_id.astype(str)

In [6]:
data.start_time = pd.to_datetime(data.start_time, format = '%Y-%m-%d')
data.stop_time = pd.to_datetime(data.stop_time, format = '%Y-%m-%d')

data['date'] = pd.to_datetime(data.start_time.dt.date)
data = data.sort_values('start_time').reset_index(drop = True)

tv_schedule = tv_schedule[tv_schedule.tv_show_id != '0']
tv_schedule['start_time'] = pd.to_datetime(tv_schedule['start_time'], format = '%d.%m.%Y %H:%M:%S')
tv_schedule = tv_schedule.sort_values('start_time').reset_index(drop = True)

In [7]:
date_list = sorted(data['date'].unique())

In [8]:
index_dict = data.groupby('user_id').apply(lambda x: x.index).to_dict()
tv_schedule_dict = tv_schedule.groupby('channel_id').apply(lambda x: x.index).to_dict()

In [9]:
tv_schedule['end_time'] = tv_schedule['start_time'] + pd.Series([np.timedelta64(x, 's') for x in tv_schedule['duration'].values])

In [10]:
schedule = tv_schedule[['start_time', 'end_time', 'tv_show_id', 'duration']].values

In [11]:
new_dict = {}
for k in tqdm(tv_schedule_dict):
    channel_index = tv_schedule_dict[k]
#     print(channel_index)
    new_dict[k] = schedule[channel_index, :]
tv_schedule_dict = dict(new_dict)
del new_dict

In [12]:
m = data[['vsetv_id', 'start_time', 'stop_time', 'duraton']].values

In [13]:
dict_cols = {'show_list':[], 'duration_ratio':[]}
for channel, start, end, duration in tqdm(m):
    if not channel in tv_schedule_dict:
        dict_cols['show_list'] += [[]] 
        dict_cols['duration_ratio'] += [[]]
        continue 
    channel_array_start = tv_schedule_dict[channel][:, 0]
    channel_array_end = tv_schedule_dict[channel][:, 1]
    show_id_array = tv_schedule_dict[channel][:, 2]
    show_duration_array = tv_schedule_dict[channel][:, 3]
    ind_first = max(np.searchsorted(channel_array_start, start, side = 'right') - 1, 0)
    ind_last = max(np.searchsorted(channel_array_start, end, side = 'right') - 1, 0)
    
    show_list = []
    duration_ratio = []
    for i in range(ind_first, ind_last + 1):
#         print(start, end)
#         print(channel_array_start[i], channel_array_end[i])
        if channel_array_start[i] <= end and channel_array_end[i] >= start:
            cur_duration = (min(channel_array_end[i], end) - max(start, channel_array_start[i])).total_seconds()
            duration_ratio += [cur_duration / show_duration_array[i]]
            show_list += [show_id_array[i]]
    dict_cols['show_list'] += [show_list] 
    dict_cols['duration_ratio'] += [duration_ratio]
#         print('HAAAI')
#     print('\n\n\n\n')

In [14]:
data['show_list'] = dict_cols['show_list']
data['duration_ratio'] = dict_cols['duration_ratio']

In [15]:
data['len_show_list'] = data['show_list'].apply(lambda x: len(x))

In [16]:
data = data[data['len_show_list'] > 0].reset_index(drop = True)

In [17]:
def map_per_image(label, predictions):   
    try:
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])

In [18]:
def create_valid(df):
    cnt_good = 0
    cnt_bad = 0
    dict_user = {'user':[], 'target_shows':[]}
    user_index_dict = df.groupby('user_id').apply(lambda x: x.index).to_dict()
    for user in tqdm(user_index_dict):
        user_shows = []
        cur_df = df.loc[user_index_dict[user], :].reset_index(drop = True)
        for show_list, ratio_list in cur_df[['show_list', 'duration_ratio']].values:
            for i in range(len(show_list)):
                if ratio_list[i] > 0.8:
                    cnt_good += 1
                    user_shows += [show_list[i]]
                else:
                    cnt_bad += 1
        user_shows = list(pd.Series(user_shows).value_counts().index)
        if len(user_shows) < 5:
            continue
        dict_user['user'] += [user]
        dict_user['target_shows'] += [user_shows[:5]]
#         print(cnt_good, cnt_bad, cnt_good / (cnt_good + cnt_bad))
    return pd.DataFrame(dict_user)

In [19]:
def get_something(df, target_dates):
    score_list = []
    start_date = target_dates[0]
    end_date = target_dates[-1]
    
    train = df[df.start_time < start_date].reset_index(drop = True)
    valid = df[(df.start_time >= start_date) & (df.start_time <= end_date)].reset_index(drop = True)
    valid = create_valid(valid)
    return train, valid

In [20]:
train, valid = get_something(data, date_list[-84:])

<ipython-input-18-b9a1df7ca65a>:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  user_shows = list(pd.Series(user_shows).value_counts().index)


In [21]:
def create_df(train, users, targets):
    df = pd.DataFrame()
    all_users_shows = []
    for show_list, ratio_list in tqdm(train[['show_list', 'duration_ratio']].values):
        for i in range(len(show_list)):
            if ratio_list[i] > 0.8:
                all_users_shows += [show_list[i]]
    all_users_shows = list(pd.Series(all_users_shows).value_counts().index)
    
    user_index_dict = train.groupby('user_id').apply(lambda x: x.index).to_dict()
    if targets is None:
        df_dict = {'user_id':[], 'tv_show_id':[]}
    else:
        df_dict = {'user_id':[], 'tv_show_id':[], 'target':[]}
    if targets is None:
        targets_new = [[0]] * len(users)
    else:
        targets_new = targets
    for user, target_list in tqdm(zip(users, targets_new)):
        target_set = set(target_list)
        if user not in user_index_dict:
            for x in all_users_shows[:30]:
                df_dict['user_id'] += [user]
                df_dict['tv_show_id'] += [x]
                if targets is None:
                    pass
                else:
                    df_dict['target'] += [1 if x in target_set else 0]
            continue
        cur_df = train.loc[user_index_dict[user], :]
        user_shows = []
        for show_list in cur_df['show_list'].values:
            for x in show_list:
                user_shows += [x]
        user_shows = set(list(np.unique(user_shows)))
        for x in user_shows:
            df_dict['user_id'] += [user]
            df_dict['tv_show_id'] += [x]
            if targets is None:
                pass
            else:
                df_dict['target'] += [1 if x in target_set else 0]
        for x in all_users_shows[:30]:
            if x not in user_shows:
                df_dict['user_id'] += [user]
                df_dict['tv_show_id'] += [x]
                if targets is None:
                    pass
                else:
                    df_dict['target'] += [1 if x in target_set else 0]
    
    return pd.DataFrame(df_dict)

In [22]:
df = create_df(train, valid.user.values, valid.target_shows.values)
df_test = create_df(data, subm.user_id.values, None)

In [23]:
channel_show_dict = tv_schedule.groupby('tv_show_id')['channel_id'].first().to_dict()

In [24]:
def create_features(train, df, date_range):
    cnt = 0
    cnt_2 = 0
    cur_schedule = tv_schedule[(tv_schedule.start_time >= date_range[0]) & (tv_schedule.start_time <= date_range[1])]
    schedule_dict = cur_schedule.tv_show_id.value_counts(normalize = True).to_dict()
    dict_ratio_features = {}
    dict_show_ratio = {}
    favorite_channel_dict = {}
    user_duration_dict = {}
    for user, channel, duration, show_list, duration_list in tqdm(train[['user_id', 'vsetv_id', 'duraton', 'show_list', 'duration_ratio']].values):
#         print(duration_list)
        if user not in dict_ratio_features:
            dict_ratio_features[user] = {}
        if user not in user_duration_dict:
            user_duration_dict[user] = []
        user_duration_dict[user] += [duration]
        
        if user not in favorite_channel_dict:
            favorite_channel_dict[user] = []
        favorite_channel_dict[user] += [channel]
        
        for i in range(len(show_list)):
            if show_list[i] not in dict_ratio_features[user]:
                dict_ratio_features[user][show_list[i]] = []
            if show_list[i] not in dict_show_ratio:
                dict_show_ratio[show_list[i]] = []
            dict_show_ratio[show_list[i]] += [duration_list[i]]
            cnt += 1
            dict_ratio_features[user][show_list[i]] += [duration_list[i]]
#     print(dict_ratio_features[user])
    dict_user_show_counts = {}
    
    for k in favorite_channel_dict:
        favorite_channel_dict[k] = pd.Series(favorite_channel_dict[k]).value_counts(normalize = True).to_dict()
    
    for k in dict_ratio_features:
        if k not in dict_user_show_counts:
            dict_user_show_counts[k] = 0
        for x in dict_ratio_features[k]:
            dict_user_show_counts[k] += len(dict_ratio_features[k][x])
    
    show_ratio_mean_dict = {}
    show_ratio_median_dict = {}
    cnt_show_ratio = {}
    all_shows = 0
    
    for k in dict_show_ratio:
        show_ratio_mean_dict[k] = np.mean(dict_show_ratio[k])
        show_ratio_median_dict[k] = np.median(dict_show_ratio[k])
        all_shows += len(dict_show_ratio[k])
        cnt_show_ratio[k] = len(dict_show_ratio[k])
    
    for k in cnt_show_ratio:
        cnt_show_ratio[k] /= all_shows
    
    dict_df_features = {'ratio_max':[], 'ratio_mean':[], 'ratio_median':[], 'ratio_min':[], 'cnt_show':[], 
                        'mean_show_ratio':[], 'median_show_ratio':[], 'cnt_show_all_normal':[], 
                       'channel_ratio':[], 'scedule_show_ratio':[], 'ratio_std':[]}
    for user, show in tqdm(df[['user_id', 'tv_show_id']].values):
        dict_df_features['scedule_show_ratio'] += [schedule_dict.get(show, 0)]
        dict_df_features['mean_show_ratio'] += [show_ratio_mean_dict.get(show, -1)]
        dict_df_features['median_show_ratio'] += [show_ratio_median_dict.get(show, -1)]
        dict_df_features['cnt_show_all_normal'] += [cnt_show_ratio.get(show, -1)]
        if user not in favorite_channel_dict:
            dict_df_features['channel_ratio'] += [-1]
        else:
            dict_df_features['channel_ratio'] += [favorite_channel_dict[user].get(channel_show_dict[show], 0)]
            
        if user not in dict_ratio_features:
            cnt += 1
            dict_df_features['ratio_max'] += [-1]
            dict_df_features['ratio_mean'] += [-1]
            dict_df_features['ratio_median'] += [-1]
            dict_df_features['ratio_min'] += [-1]
            dict_df_features['ratio_std'] += [-1]
            dict_df_features['cnt_show'] += [-1]
            continue
        if show not in dict_ratio_features[user]:
            cnt_2 += 1
            dict_df_features['ratio_max'] += [-1]
            dict_df_features['ratio_mean'] += [-1]
            dict_df_features['ratio_median'] += [-1]
            dict_df_features['ratio_min'] += [-1]
            dict_df_features['ratio_std'] += [-1]
            dict_df_features['cnt_show'] += [-1]
            continue
        
        dict_df_features['ratio_max'] += [np.max(dict_ratio_features[user][show])]
        dict_df_features['ratio_std'] += [np.std(dict_ratio_features[user][show])]
        dict_df_features['ratio_mean'] += [np.mean(dict_ratio_features[user][show])]
        dict_df_features['ratio_median'] += [np.median(dict_ratio_features[user][show])]
        dict_df_features['ratio_min'] += [np.min(dict_ratio_features[user][show])]
        dict_df_features['cnt_show'] += [len(dict_ratio_features[user][show]) / (dict_user_show_counts[user] + 1)]
    print(cnt, cnt_2)
    return pd.concat([df, pd.DataFrame(dict_df_features)], axis = 1)

In [25]:
df = create_features(train, df, (date_list[-84:][0], date_list[-84:][-1]))
df_test = create_features(data, df_test, (date_list[-84:][-1], date_list[-84:][-1] + np.timedelta64(365, 'D')))


1906728 61494



3848927 15267


In [26]:
ltr = len(df)

In [27]:
df = pd.concat([df, df_test], axis = 0).reset_index(drop = True)

In [28]:
df.target.value_counts()

0.0    675736
1.0     15449
Name: target, dtype: int64

In [29]:
df['show_duraion_mean'] = df['tv_show_id'].map(
    tv_schedule.groupby('tv_show_id')['duration'].mean().to_dict()).fillna(0)
df['show_duraion_median'] = df['tv_show_id'].map(
    tv_schedule.groupby('tv_show_id')['duration'].median().to_dict()).fillna(0)
df['show_duraion_max'] = df['tv_show_id'].map(
    tv_schedule.groupby('tv_show_id')['duration'].max().to_dict()).fillna(0)
df['show_duraion_min'] = df['tv_show_id'].map(
    tv_schedule.groupby('tv_show_id')['duration'].min().to_dict()).fillna(0)
df['show_duraion_std'] = df['tv_show_id'].map(
    tv_schedule.groupby('tv_show_id')['duration'].std().to_dict()).fillna(0)

In [31]:
def standart_split(data, ltr):
    split_list = []
    kf = GroupKFold(n_splits = 10)
    for train_index, test_index in kf.split(data.loc[:ltr-1, :], data.loc[:ltr-1, 'target'], data.loc[:ltr-1, 'user_id']):
        split_list += [(train_index, test_index)]
    return split_list
split_list = standart_split(df, ltr)

In [32]:
def lgb_train(data, target, ltr, split_list, param, v_e = 0, n_e = 20000, cat_col = None):
    pred = pd.DataFrame()
    pred_val = np.zeros(ltr)
    fi = np.zeros(data.shape[1])
    score = []
    bst_list = []
    pred_train = pd.DataFrame()
    j = 0
    target = target
    for i , (train_index, test_index) in enumerate(split_list):
        tr = lgb.Dataset(np.array(data)[train_index], np.array(target)[train_index])
        te = lgb.Dataset(np.array(data)[test_index], np.array(target)[test_index], reference=tr)
        evallist = [(tr, 'train'), (te, 'test')]
        bst = lgb.train(param, tr, num_boost_round=n_e, 
                valid_sets = [tr, te], early_stopping_rounds=int(5 / param['learning_rate']), verbose_eval = v_e)
#         return bst
        pred[str(i)] = bst.predict(np.array(data)[ltr:, :])
        pred_val[test_index] = bst.predict(np.array(data)[test_index]) 
        cur_score = roc_auc_score(np.array(target)[test_index], pred_val[test_index])
        score += [cur_score]
        bst_list += [bst]
        fi += np.array(bst.feature_importance(importance_type = 'gain'))
        print(i+1, np.mean(score))
        print()
        del tr, te
    pred_train[str(j)] = pred_val
    return bst_list, pred_train, pred, score, fi

In [45]:
param_lgb = {
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'boost': 'gbdt',
    'feature_fraction': 0.9,
    'learning_rate': 0.05,
    'metric':'auc',
    'num_leaves': 8,
#     'min_data_in_leaf':1,
    'objective': 'binary',
    'lambda_l1':5,
    'verbose':-1
    }
    
    

In [46]:
train_cols = ['ratio_max', 'ratio_mean',
       'ratio_median', 'ratio_min', 'cnt_show', 'show_duraion_mean',
       'show_duraion_median', 'show_duraion_max', 'show_duraion_min', 'channel_ratio', 'scedule_show_ratio',
             'show_duraion_std', 'ratio_std'] 

In [47]:
sorted(zip(fi, train_cols))

[(16717.13015642017, 'ratio_median'),
 (21871.87955148518, 'ratio_min'),
 (33095.837028704584, 'show_duraion_max'),
 (35092.18068858236, 'ratio_mean'),
 (41273.551631361246, 'ratio_std'),
 (47891.97073931992, 'show_duraion_median'),
 (48659.27051529288, 'show_duraion_std'),
 (64045.320051796734, 'channel_ratio'),
 (68343.10505642742, 'show_duraion_min'),
 (206919.31691598892, 'ratio_max'),
 (756622.0571559891, 'show_duraion_mean'),
 (1040112.8389275968, 'scedule_show_ratio'),
 (1460015.025303021, 'cnt_show')]

In [48]:
bst_list, pred_train, pred, score, fi = lgb_train(df[train_cols], df['target'], ltr, split_list, param_lgb, 50)

Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.958769	valid_1's auc: 0.958712
[100]	training's auc: 0.961802	valid_1's auc: 0.960854
[150]	training's auc: 0.963297	valid_1's auc: 0.96215
[200]	training's auc: 0.964135	valid_1's auc: 0.96284
[250]	training's auc: 0.964788	valid_1's auc: 0.963265
[300]	training's auc: 0.96526	valid_1's auc: 0.963597
[350]	training's auc: 0.965659	valid_1's auc: 0.963771
[400]	training's auc: 0.966013	valid_1's auc: 0.963953
[450]	training's auc: 0.966299	valid_1's auc: 0.964098
[500]	training's auc: 0.96654	valid_1's auc: 0.964234
[550]	training's auc: 0.966765	valid_1's auc: 0.964343
[600]	training's auc: 0.967007	valid_1's auc: 0.964357
[650]	training's auc: 0.967225	valid_1's auc: 0.964504
[700]	training's auc: 0.967409	valid_1's auc: 0.964509
[750]	training's auc: 0.967575	valid_1's auc: 0.964534
[800]	training's auc: 0.967806	valid_1's auc: 0.964599
[850]	training's auc: 0.967982	valid_1's auc: 0.964603
[900]	tr

[300]	training's auc: 0.965416	valid_1's auc: 0.962544
[350]	training's auc: 0.965771	valid_1's auc: 0.962764
[400]	training's auc: 0.966122	valid_1's auc: 0.963005
[450]	training's auc: 0.966422	valid_1's auc: 0.963164
[500]	training's auc: 0.966693	valid_1's auc: 0.963362
[550]	training's auc: 0.966914	valid_1's auc: 0.963433
[600]	training's auc: 0.96717	valid_1's auc: 0.963519
[650]	training's auc: 0.967346	valid_1's auc: 0.96354
[700]	training's auc: 0.967515	valid_1's auc: 0.963627
[750]	training's auc: 0.967662	valid_1's auc: 0.963637
[800]	training's auc: 0.967837	valid_1's auc: 0.963646
[850]	training's auc: 0.967994	valid_1's auc: 0.96367
[900]	training's auc: 0.968148	valid_1's auc: 0.963688
[950]	training's auc: 0.96831	valid_1's auc: 0.963718
[1000]	training's auc: 0.968452	valid_1's auc: 0.963706
[1050]	training's auc: 0.968608	valid_1's auc: 0.963727
[1100]	training's auc: 0.968742	valid_1's auc: 0.963749
[1150]	training's auc: 0.968893	valid_1's auc: 0.963778
[1200]	tra

In [ ]:
5 0.9615233922731707
0.25372059529524726

In [49]:
train_data = df.loc[:ltr-1, :].reset_index(drop = True)

train_data['pred'] = pred_train['0']

In [50]:
pred_dict = train_data.sort_values('pred', ascending = False).groupby('user_id')['tv_show_id'].apply(
    lambda x: list(x)[:5]).to_dict()

In [51]:
valid['pred'] = valid['user'].map(pred_dict)

In [52]:
score_list = []
for x,y in valid[['target_shows', 'pred']].values:
    score = map_per_set(x, [y] * 5)
    score_list += [score]

In [53]:
np.mean(score_list)

0.25776684269483113

In [58]:
test = df.loc[ltr:, :].reset_index(drop = True)
test['pred'] = pred.mean(axis = 1).values
pred_test_dict = test.sort_values('pred', ascending = False).groupby('user_id')['tv_show_id'].apply(
    lambda x: ' '.join([str(x) for x in list(x)[:5]])).to_dict()

In [59]:
subm.tv_show_id = subm.user_id.map(pred_test_dict)

In [60]:
subm.to_csv('loh_lohovich_2.csv', index = None)

In [61]:
subm

,user_id,tv_show_id
0,8377619604347126107,2400480 240081 2400508 20088 12002955
1,8381667675275833309,2400480 240081 12001682 12002856 12002355
2,8387147770138767246,12001682 12002856 12002955 12002355 12001732
3,8397181578236218580,2400480 20088 240081 2400467 6700482
4,8404698046253197367,2400480 240081 20088 12001682 6700482
...,...,...
1255,10876606683805171865,2400480 20088 240081 12001682 12001712
1256,10886096340955133676,12001682 12002856 12001734 2400480 10001514
1257,10902911550446151928,240081 12001682 2400480 12002856 12001734
1258,10904697592493678571,12001682 240081 2400480 12001734 12002856


In [ ]:
0.22139062249959993

In [ ]:
user_index_dict = data.groupby('user_id').apply(lambda x: x.index).to_dict()

In [ ]:
all_users_shows = []
for show_list, ratio_list in tqdm(data[['show_list', 'duration_ratio']].values):
    for i in range(len(show_list)):
        if ratio_list[i] > 0.8:
            all_users_shows += [show_list[i]]
all_users_shows = list(pd.Series(all_users_shows).value_counts().index)

In [ ]:
set_last_shows = set(tv_schedule[tv_schedule.start_time > pd.to_datetime('2020-07-27', 
                                                                         format = '%Y-%m-%d')].tv_show_id.unique())

In [ ]:
answer_list = []

for user in tqdm(subm.user_id.values):
    answer = []
    if not user in user_index_dict:
        for x in all_users_shows:
            if (x not in answer) and (x in set_last_shows):
                answer += [x]
                if len(answer) == 5:
                    break
        answer_list += [' '.join([str(x) for x in answer])]
        continue
    cur_data = data.loc[user_index_dict[user], :]
    user_good_shows = []
    for show_list, ratio_list in cur_data[['show_list', 'duration_ratio']].values:
        for i in range(len(show_list)):
            if ratio_list[i] > 0.8:
                user_good_shows += [show_list[i]]
    for x in pd.Series(user_good_shows).value_counts().index:
        if (x not in answer) and (x in set_last_shows):
            answer += [x]
            if len(answer) == 5:
                break
    if len(answer) == 5:
        answer_list += [' '.join([str(x) for x in answer])]
        continue
    for x in all_users_shows:
        if (x not in answer) and (x in set_last_shows):
            answer += [x]
            if len(answer) == 5:
                break
    if len(answer) == 5:
        answer_list += [' '.join([str(x) for x in answer])]
        continue

In [ ]:
subm['tv_show_id'] = answer_list

In [ ]:
subm.to_csv('therealroman_2.csv', index = None)

In [ ]:
subm

In [ ]:
gg = pd.read_csv('therealroman.csv')

In [ ]:
all_score = 0
for x, y in zip(gg.tv_show_id.values, subm.tv_show_id.values):
    score = len(set(x.split(' ')) & set(y.split(' ')))
    score = 5 - score
    all_score += score

In [ ]:
tv_schedule[tv_schedule.channel_id == 1174]

In [ ]:
set(data.vsetv_id.values) - set(tv_schedule.channel_id.values)